In [1]:
#Imports

import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers, callbacks
print(tf.__version__)


2.6.0


My First Visual Studio Code, Tensorflow, & Jupyter Notebook Project

Code copied from https://www.kaggle.com/ryanholbrook/custom-convnets

In [94]:
#import and process annotations
cars_meta = scipy.io.loadmat('./Annos/cars_meta.mat')
class_names = cars_meta['class_names']  # shape=(1, 196)
class_names = np.transpose(class_names)

train_annos = scipy.io.loadmat('./Annos/cars_train_annos.mat')
train_annos = train_annos['annotations']
train_annos = np.transpose(train_annos)

test_annos = scipy.io.loadmat('./Annos/cars_test_annos_withlabels.mat')
test_annos = test_annos['annotations']
test_annos = np.transpose(test_annos)

def format_annotations(data):

    annos = []
 
    for annotation in data:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        class_id = annotation[0][4][0][0]
        fname = annotation[0][5][0]
        annos.append([fname,[bbox_x1, bbox_y1, bbox_x2, bbox_y2],class_id])
    
    return(annos)

train_annotations = format_annotations(train_annos)
test_annotations = format_annotations(test_annos)

#get annotations train_annotations[0][2]. First index is the number of images.  Second index is [0] for frame name. [1] for box. [2] for class_id

#save labels as list
def labels_list(data):

    labels = []

    for label in data:
        labels.append(label[2])

    return(labels)

def fnames_list(data):

    fnames = []

    for fname in data:
        fnames.append(fname[0])
    
    return(fnames)

train_labels = labels_list(train_annotations)
test_labels = labels_list(test_annotations)


train_fnames = fnames_list(train_annotations)
test_fnames = fnames_list(test_annotations)



8144


In [44]:
print(train_annotations[8143])
print(train_annotations[0][1])
index = 0
x1 = train_annotations[index][1][0]
y1 = train_annotations[index][1][1]
x2 = train_annotations[index][1][2]
y2 = train_annotations[index][1][3]
print(x1,y1,x2,y2)

print(test_annotations[0])
print(test_annotations[0][1])
index = 0
x1 = test_annotations[index][1][0]
y1 = test_annotations[index][1][1]
x2 = test_annotations[index][1][2]
y2 = test_annotations[index][1][3]
print(x1,y1,x2,y2)

['08144.jpg', [20, 240, 862, 677], 17]
[39, 116, 569, 375]
39 116 569 375
['00001.jpg', [30, 52, 246, 147], 181]
[30, 52, 246, 147]
30 52 246 147


In [18]:
#**Crop to bounding box.  Only run with full resolution images.**

# import PIL
# import os
# import os.path
# from PIL import Image

# f = r'./cars196_train'
# #f = r'./cars196_test'
# index = 0

# for file in sorted(os.listdir(f)):

#     # x1 = test_annotations[index][1][0]
#     # y1 = test_annotations[index][1][1] 
#     # x2 = test_annotations[index][1][2]
#     # y2 = test_annotations[index][1][3]

#     x1 = train_annotations[index][1][0]
#     y1 = train_annotations[index][1][1] 
#     x2 = train_annotations[index][1][2]
#     y2 = train_annotations[index][1][3]

    
    
#     f_img = f+"/"+file
#     print(f_img)
#     if(file != '.DS_Store'):
#         img = Image.open(f_img)
#         img = img.crop((x1,y1,x2,y2))
#         img.save(f_img)
#         index = index + 1

./cars196_train/.DS_Store
./cars196_train/00001.jpg
./cars196_train/00002.jpg
./cars196_train/00003.jpg
./cars196_train/00004.jpg
./cars196_train/00005.jpg
./cars196_train/00006.jpg
./cars196_train/00007.jpg
./cars196_train/00008.jpg
./cars196_train/00009.jpg
./cars196_train/00010.jpg
./cars196_train/00011.jpg
./cars196_train/00012.jpg
./cars196_train/00013.jpg
./cars196_train/00014.jpg
./cars196_train/00015.jpg
./cars196_train/00016.jpg
./cars196_train/00017.jpg
./cars196_train/00018.jpg
./cars196_train/00019.jpg
./cars196_train/00020.jpg
./cars196_train/00021.jpg
./cars196_train/00022.jpg
./cars196_train/00023.jpg
./cars196_train/00024.jpg
./cars196_train/00025.jpg
./cars196_train/00026.jpg
./cars196_train/00027.jpg
./cars196_train/00028.jpg
./cars196_train/00029.jpg
./cars196_train/00030.jpg
./cars196_train/00031.jpg
./cars196_train/00032.jpg
./cars196_train/00033.jpg
./cars196_train/00034.jpg
./cars196_train/00035.jpg
./cars196_train/00036.jpg
./cars196_train/00037.jpg
./cars196_tr

In [112]:
# Reproducability
# Setup the random seed so training data is feed in the same each run.
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
# The plotting layout presets.
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')

directory_train = './cars196_train/'
directory_test = './cars196_test/'


# Create Datasets
ds_train = tf.data.Dataset.from_tensor_slices((train_fnames, train_labels))
# ds_test = tf.data.Dataset.from_tensor_slices((test_fnames, test_labels))

def train_read_image(image_file, label):
    image = tf.io.read_file(directory_train + image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    return image, label

def test_read_image(image_file, label):
    image = tf.io.read_file(directory_test + image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    return image, label


def augment(image_file, label):
    image_size = [128,128]
    num_channels = 3
    interpolation = 'nearest'
    img = tf.image.resize(image_file, image_size, method=interpolation)
    img.set_shape((image_size[0], image_size[1], num_channels))
    return img, label


def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

# ds_train = train_read_image(train_fnames, train_labels)

def load_images(data):

    images = []

    for image in data:
        image = tf.io.read_file(directory_train + data)
        image = tf.image.decode_jpeg(image, channels=3)
        images.append(image)

    return images


ds_train_images = load_images(train_fnames)



# Putting it all together.  Take the training dataset which is sized and labeled.  Convert to pixel array.  Cache in memory for faster runtime.  Autotune sets up the CPU so it's fetching the next image in the list while the current image is in the CNN.  
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# ds_train = (ds_train.map(train_read_image).map(augment).map(convert_to_float).batch(64)).shuffle(100).cache().prefetch(buffer_size=AUTOTUNE))
# ds_test = (ds_test.map(test_read_image).map(augment).map(convert_to_float).batch(64)).cache().prefetch(buffer_size=AUTOTUNE))
# ds_train = ds_train.map(read_image).map(augment).batch(64)



# for x,y in ds_train:
#     print(x,y)


ValueError: Value tf.Tensor(b'00001.jpg,00002.jpg,00003.jpg,00004.jpg,00005.jpg,00006.jpg,00007.jpg,00008.jpg,00009.jpg,00010.jpg,00011.jpg,00012.jpg,00013.jpg,00014.jpg,00015.jpg,00016.jpg,00017.jpg,00018.jpg,00019.jpg,00020.jpg,00021.jpg,00022.jpg,00023.jpg,00024.jpg,00025.jpg,00026.jpg,00027.jpg,00028.jpg,00029.jpg,00030.jpg,00031.jpg,00032.jpg,00033.jpg,00034.jpg,00035.jpg,00036.jpg,00037.jpg,00038.jpg,00039.jpg,00040.jpg,00041.jpg,00042.jpg,00043.jpg,00044.jpg,00045.jpg,00046.jpg,00047.jpg,00048.jpg,00049.jpg,00050.jpg,00051.jpg,00052.jpg,00053.jpg,00054.jpg,00055.jpg,00056.jpg,00057.jpg,00058.jpg,00059.jpg,00060.jpg,00061.jpg,00062.jpg,00063.jpg,00064.jpg,00065.jpg,00066.jpg,00067.jpg,00068.jpg,00069.jpg,00070.jpg,00071.jpg,00072.jpg,00073.jpg,00074.jpg,00075.jpg,00076.jpg,00077.jpg,00078.jpg,00079.jpg,00080.jpg,00081.jpg,00082.jpg,00083.jpg,00084.jpg,00085.jpg,00086.jpg,00087.jpg,00088.jpg,00089.jpg,00090.jpg,00091.jpg,00092.jpg,00093.jpg,00094.jpg,00095.jpg,00096.jpg,00097.jpg,

In [69]:

# # Reproducability
# # Setup the random seed so training data is feed in the same each run.
# def set_seed(seed=31415):
#     np.random.seed(seed)
#     tf.random.set_seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
# set_seed()

# # Set Matplotlib defaults
# # The plotting layout presets.
# plt.rc('figure', autolayout=True)
# plt.rc('axes', labelweight='bold', labelsize='large',
#        titleweight='bold', titlesize=18, titlepad=10)
# plt.rc('image', cmap='magma')


# #Create a tensorflow datasta (tf.data.Dataset).  Matches the images with the categorical label.  All the images are 128x128 and if they need to be resized use nearsest neighbor interpolation.    Shuffle the training set.  Do not shuffle the validation set.  It doesn't matter the order of the validation no need to shuffle. 
# ds_train_ = tf.keras.preprocessing.image_dataset_from_directory(
#     directory='./cars196_train/',
#     labels=train_labels,
#     #Changed from binary to categorical_crossentropy because of the expanded labels.
#     label_mode='categorical',
#     image_size=[128, 128],
#     interpolation='nearest',
#     batch_size=64,
#     shuffle=True,
# )
# ds_valid_ = tf.keras.preprocessing.image_dataset_from_directory(
#     directory='./cars196_test/',
#     labels=test_labels,
#     #Changed from binary to categorical_crossentropy because of the expanded labels.
#     label_mode='categorical',
#     image_size=[128, 128],
#     interpolation='nearest',
#     batch_size=64,
#     shuffle=False,
# )

# # Data Pipeline
# # Process the images into pixel arrays so matrix operations can be preformed on them.  
# def convert_to_float(image, label):
#     image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     return image, label

# # Putting it all together.  Take the training dataset which is sized and labeled.  Convert to pixel array.  Cache in memory for faster runtime.  Autotune sets up the CPU so it's fetching the next image in the list while the current image is in the CNN.  
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# ds_train = (
#     ds_train_
#     .map(convert_to_float)
#     .cache()
#     .prefetch(buffer_size=AUTOTUNE)
# )
# ds_valid = (
#     ds_valid_
#     .map(convert_to_float)
#     .cache()
#     .prefetch(buffer_size=AUTOTUNE)
# )



Load Pretrained Base

In [ ]:
#VGG16 pretrained base for baseline.

pretrained_base = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=[128,128,3],
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
pretrained_base.trainable = False

Early Stopping

In [82]:
# Stop early if the accucary is not improving enough.

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

Attach Head

In [88]:




# Pretrained base model

# model = keras.Sequential([
#     pretrained_base,
#     layers.Flatten(),
#     layers.Dense(6, activation='relu'),
#     layers.Dense(1, activation='sigmoid'),
# ])

# Custom base  0.8051346 val_binary_accuracy

model = keras.Sequential([

    # First Convolutional Block
    # 32 filter layers, Kernel Size of 5 x 5. Relu activation.  Add zeroes all around so the image doesn't change size, Padding='same'.
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
                  input_shape=[128, 128, 3]),
    layers.Dropout(0.1),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.Dropout(0.1),
    layers.MaxPool2D(),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.Dropout(0.1),
    layers.MaxPool2D(),

    #Fourth Convolutional Block
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding='same'),
    layers.Dropout(0.1),
    layers.MaxPool2D(),

    # Classifier Head.  Fully connected Dense layer with 6 nodes and a relu activation.  Final node for binary decision. 
    layers.Flatten(),
    layers.Dense(units=392, activation="relu"),
    layers.Dense(units=196, activation="Softmax"),
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
dropout_8 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 128)      

Train

In [90]:
#Compile.  Use the Adam optimizer which uses stochastic gradient descent to adjust weights.  Binary_crossentropy since it's either 'car' or 'truck.

model.compile(
    optimizer='adam',
    loss='CategoricalCrossentropy',
    metrics=['CategoricalCrossentropy'],
)

# Fit the Model. 
history = model.fit(
    ds_train,
    validation_data = ds_test,
    epochs = 50,
    callbacks=[early_stopping], 
    verbose=1,
)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'PrefetchDataset'

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();
print(history_frame.val_binary_accuracy)
